In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb

In [150]:
#    Dictionaries and lists
important=['Participant ID','Date of Intake','VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
       'VEG0600','VEG0900','VEG0500','VEG0100','VEG0700','FRU0100','FRU0200','FRU0300','FRU0400',
       'FRU0500','FRU0600','FRU0700','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
       'Whole Grains (ounce equivalents)','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
       'DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500',
       'DOT0600','DOT0100','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
       'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0100','MFL0100',
       'MFF0200','MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400',
       'MOF0500','MOF0600','MOF0700','VEG0700','MFF0100','MFL0100','MFF0200','MSL0100','MSF0100',
       'MOF0500','MOF0600','MOF0700','VEG0700','Sodium (mg)','Refined Grains (ounce equivalents)',
       'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
       'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
       'Total Saturated Fatty Acids (SFA) (g)','FRU0100','FRU0200','DMF0200','DMR0200','DML0200','DML0300',
       'SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600','SWT0100',
       'SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400','SWT0500','DOT0300' , 'DOT0400',
       'DOT0100','DOT0200','GRR0800' , 'GRS0800',  'GRW0800','GRW0900' ,'GRS0900' , 'GRR0900' ,
       'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400','VEG0800','FMC0100', 'FMC0200','DMF0100',
       'DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600','DOT0700',
       'DOT0800','GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700','GRW1300','GRS1300']


ped811_dict = {
    'hei_vegetables': {'parameters':[0.1,1.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,1.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.0,3.5,0.0 , 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[20,28,8,35], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.5,6.0,0,10.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.6,3.5], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
    }

ped1224_dict = {
    'hei_vegetables': {'parameters':[0.1,7.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,7.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.5, 5.5, 0, 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[14.0,18.0,8.0,24.0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.0,3.0,0,6.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.9, 4.2], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[4.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
    }

ped08_dict = {
    'hei_vegetables': {'parameters':[0,0], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0,0], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[0,0,0,0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_dairy': {'parameters':[0,0,0,0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[0,0,0,0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[0,0], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0,0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0,0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0,0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0,0], 'name': 'HEIX10_SALTY'},
    'hei_cereal': {'parameters':[0,0], 'name': 'HEIX11_CEREAL'}
    }

ped_dict={
    'young': ped811_dict,
    'child':ped1224_dict,
    'infant':ped08_dict
    }

hei_ped_dict={
        'hei_totveg':
        ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
        'hei_greensbeans':
        ['VEG0100','VEG0700'],
        'hei_totfruit':
        ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholefruit':
        ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholegrains':
        ['Whole Grains (ounce equivalents)'],
        'hei_dairy':
        ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
        'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
        'hei_totproteins':
        ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
        'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
        'MFF0100','MFL0100','MFF0200','MSL0100',
        'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
        'MOF0600','MOF0700','VEG0700'],
        'hei_seafoodplantprot':
        ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
        'hei_refinedgrains':
        ['Refined Grains (ounce equivalents)'],
        'energy':
        ['Energy (kcal)'],
        'hei_fruitjuice':
        ['FRU0100','FRU0200'],
        'hei_SSB':
        ['DMF0200','DMR0200','DML0200','DML0300','SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600'],
        'chocolate_candies':
        ['SWT0100'],
        'candies':
        ['SWT0200'],
        'frosting':
        ['SWT0300'],
        'sweet_sauce':
        ['SWT0700' , 'SWT0800'],
        'sugar':
        ['SWT0400'],
        'syrups':
        ['SWT0500'],
        'Pudding':
        ['DOT0300' , 'DOT0400'],
        'icecream':
        ['DOT0100' ],
        'nondairy_treat':
        ['DOT0200' ],
        'baked_good':
        ['GRR0800' , 'GRS0800',  'GRW0800' ],
        'chips':
        ['GRW0900' ,'GRS0900' , 'GRR0900' , 'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400' ],
        'fries':
        ['VEG0800' ],
        'other_fried':
        ['FMC0100', 'FMC0200' ],
        'milk':
        ['DMF0100','DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600'],
        'formula_foz':
        ['DOT0700', 'DOT0800'],
        'cereal_oz':
        ['GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700'],
        'bbcereal_hcup':
        ['GRW1300','GRS1300']
        }

conv_dict={
'hei_fruitjuice':4,
'hei_SSB':8,
'chocolate_candies':40,
'candies':15,
'frosting':35,
'sugar':4,
'syrups':.25,
'Pudding':1,
'icecream':.5,
'nondairy_treat':85,
'baked_good':55,
'fries': 70,
'formula_foz':5,
'bbcereal_hcup':.5
}

make_hei_dict={
'hei_sweets' :
['chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good'],
'hei_salty' :
['chips','other_fried','fries'],
'hei_vegetables':
['hei_totveg', 'hei_greensbeans'],
'hei_proteins':
['hei_totproteins','hei_seafoodplantprot'],
'hei_milk':
['hei_dairy','formula_foz'],
'hei_cereal':
['cereal_oz', 'bbcereal_hcup']
}

ped_interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot',
            'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good',
            'chips','other_fried','fries']

In [151]:
basepath='/Users/gracer/Google Drive/BCP/data'
arglist={'BASEPATH': basepath, 
         'CHILD': True, 
         'NAMES': 'testies', 
         'OPTS': ['Chil'], 
         'SAVE':os.path.join(basepath, 'derivived'),
         'XTRA': 'total_demo_dataset.csv'}

In [152]:
# Find the data
for (dirpath, dirnames, filenames) in os.walk(basepath):
   for filename in filenames:
       if filename.endswith('.zip'):
           tmppath=os.sep.join([dirpath, filename])
           with ZipFile(tmppath, 'r') as zipObj:
              # Get a list of all archived file names from the zip
              listOfFileNames = zipObj.namelist()
              # Iterate over the file names
              for fileName in listOfFileNames:
                  # Check filename endswith txt
                   if fileName.endswith('04.txt'):
                       zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                   if fileName.endswith('09.txt'):
                       zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')


In [153]:
infile

'/Users/gracer/Google Drive/BCP/data/temp_txt'

In [154]:
def file_org(infile, arglist):
    # will create a dictionary with the file paths to all the data
    if arglist['OPTS'] == False:
        arglist['OPTS']=[arglist['NAMES']]
        file_dict = {"set_04": {"%s"%arglist['NAMES']:{}}, "set_09": {"%s"%arglist['NAMES']: {}}}
        file_dict['set_04']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "%s"%arglist['NAMES'] in x]
        file_dict['set_09']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "%s"%arglist['NAMES'] in x]
        arglist['OPTS']=[arglist['NAMES']]

    else:
        file_dict={"set_04":{},"set_09":{}}
        for item in arglist['OPTS']:
            print('this is the item %s'%item)
            file_dict["set_04"][item]= {}
            file_dict["set_09"][item]= {}

        for key,value in file_dict.items():
            keys=list(file_dict[key].keys())
            for k in keys:
                print(k)
                file_dict['set_04']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if '%s'%k in x]
                file_dict['set_09']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if '%s'%k in x]

    return(file_dict)


In [155]:
a=file_org(infile, arglist) #list of the files


this is the item Chil
Chil
Chil


In [156]:
demo_interest = ["child_feeding_practice-age_any_formula", "child_feeding_practice-age_any_formula_status", "child_feeding_practice-age_fed", "child_feeding_practice-age_fed_dropdown",	"child_feeding_practice-age_fed_status", "child_feeding_practice-age_regular_formula",	"child_feeding_practice-age_regular_formula_status", "child_feeding_practice-age_stop", "child_feeding_practice-age_stop_dropdown", "child_feeding_practice-age_stop_formula", "child_feeding_practice-age_stop_formula_dropdown","child_feeding_practice-age_stop_formula_status","child_feeding_practice-age_stop_status","child_feeding_practice-any_formula",	"child_feeding_practice-birth_order",	"child_feeding_practice-breastfed",	"child_feeding_practice-four_month_ratio",	"child_feeding_practice-regular_formula", "Identifiers_visit", "Date_taken","weight","head_circumference","height","Participant ID","Gender","DoB"]


In [157]:
# proto file reader
data_dict={"set_04":{},"set_09":{},"demo":{}}
df_demo=pd.read_csv(os.path.join(basepath,arglist['XTRA']), sep=",")

df_demo['Participant ID'] = df_demo['Participant ID'].astype('int32')
df_demo['Identifiers_visit']=df_demo['Identifiers_visit'].astype('int32')
df_demo['child_feeding_practice-breastfed']=df_demo['child_feeding_practice-breastfed'].astype('category')

data_dict["demo"]= df_demo[demo_interest]

for key, value in a.items():
    print(key)
    for k,v, in value.items():
        print(k)
        temp_list = []
        for file in v["files"]:
            print(file)
            temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
            if key == 'set_09':
                temp_df=temp_df.drop([0]) #drops extra row 
            for val in temp_df["Participant ID"]:
                _id = str(val).lstrip("0").split("_")[0]
                temp_df.replace(val, _id, inplace=True)
            temp_list.append(temp_df)
        dfm_original = pd.concat(temp_list, ignore_index=True)
        print("Final dataframe size: ", dfm_original.shape)
        dfm_original = dfm_original.sort_values(by="Participant ID")
         
        del_cols=set(dfm_original.columns) - set(important)
        dfm_original.drop(del_cols, axis=1, inplace=True)
        for col in dfm_original:
            if dfm_original[col].dtype == np.object_:
                dfm_original[col] = (dfm_original[col].replace(',','.', regex=True))   
        
        concat_filepath = os.path.join(arglist['SAVE'],'%s_dataset_%s.csv'%(k,key))

        dfm_original['Participant ID'] = dfm_original['Participant ID'].astype('float').astype('int32')
        dfm_original.sort_values("Participant ID", inplace=True) 

        dfm_original.drop_duplicates(keep='first',inplace=True) 
        
       
        _sub=pd.merge(data_dict['demo'][['Participant ID','DoB']], dfm_original[['Participant ID','Date of Intake']],on='Participant ID')
        _sub['age at intake']=ager(_sub['Date of Intake'], _sub['DoB'])
        grp=['Participant ID','Date of Intake']
        dfm_original=pd.merge(dfm_original,_sub, on=grp)
        metric_cols=list(set(dfm_original.columns) - set(grp))
        print(metric_cols)
        aggs = dfm_original.groupby(grp)[metric_cols].mean()
        dfm_original.drop(metric_cols, axis=1, inplace=True)
        # dedupe to leave only one row with each combination of group_cols
        # in df
        dfm_original.drop_duplicates(subset=grp, keep='last', inplace=True)
        # add the mean columns from aggs into df
        dfm_original = dfm_original.merge(right=aggs, right_index=True, left_on=grp, how='right')

        
        data_dict[key]= dfm_original


set_04
Chil
/Users/gracer/Google Drive/BCP/data/temp_txt/UNCChildJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP04.txt
Final dataframe size:  (938, 226)
['Sodium (mg)', 'Added Sugars (by Total Sugars) (g)', 'Total Polyunsaturated Fatty Acids (PUFA) (g)', 'Total Monounsaturated Fatty Acids (MUFA) (g)', 'Refined Grains (ounce equivalents)', 'Energy (kcal)', 'Total Saturated Fatty Acids (SFA) (g)', 'DoB', '% Calories from SFA', 'Whole Grains (ounce equivalents)', 'age at intake']
set_09
Chil
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt

In [160]:
data_dict['set_09']

,Participant ID,Date of Intake,age at intake
2,2081,08/12/2017,0.788517
5,2081,10/25/2018,15.211811
8,2081,08/03/2017,0.492823
14,11228,09/09/2018,18.792994
20,11228,02/13/2018,11.959178
...,...,...,...
2495,993015,07/18/2018,14.751843
2500,993015,07/22/2018,14.883262
2505,993015,01/05/2018,8.377995
2507,999647,04/13/2018,16.033183


In [ ]:
# identify the columns we want to aggregate by; this could
# equivalently be defined as list(df.columns[0:3])
group_cols = ['tracking_id', 'gene_id', 'gene_short_name']
# identify the columns which we want to average; this could
# equivalently be defined as list(df.columns[4:])
metric_cols = ['FPKM-1', 'FPKM-2']

# create a new DataFrame with a MultiIndex consisting of the group_cols
# and a column for the mean of each column in metric_cols
aggs = df.groupby(group_cols)[metric_cols].mean()
# remove the metric_cols from df because we are going to replace them
# with the means in aggs
df.drop(metric_cols, axis=1, inplace=True)
# dedupe to leave only one row with each combination of group_cols
# in df
df.drop_duplicates(subset=group_cols, keep='last', inplace=True)
# add the mean columns from aggs into df
df = df.merge(right=aggs, right_index=True, left_on=group_cols, how='right')

In [141]:
data_dict['set_09'].shape

(603, 1)

In [135]:
dfm_original=data_dict['set_04']
df_demo=pd.read_csv(os.path.join(basepath,arglist['XTRA']), sep=",")

df_demo['Participant ID'] = df_demo['Participant ID'].astype('int32')
df_demo['Identifiers_visit']=df_demo['Identifiers_visit'].astype('int32')
df_demo['child_feeding_practice-breastfed']=df_demo['child_feeding_practice-breastfed'].astype('category')

data_dict["demo"]= df_demo[demo_interest]
_sub=pd.merge(data_dict['demo'][['Participant ID','DoB']], dfm_original[['Participant ID','Date of Intake']],on='Participant ID')
_sub['age at intake']=ager(_sub['Date of Intake'], _sub['DoB'])
grp=['Participant ID','Date of Intake']
dfm_original=pd.merge(dfm_original,_sub, on=grp)
metric_cols=set(dfm_original.columns) - set(grp)
dfm_original = dfm_original.groupby(grp)[list(metric_cols)].mean()


KeyError: "None of [Index(['Participant ID', 'Date of Intake'], dtype='object')] are in the [columns]"

In [131]:
dfm_original

Sodium (mg)  \
Participant ID Date of Intake                
2081           08/03/2017          116.867   
               08/12/2017          220.536   
               10/25/2018          313.230   
11228          02/11/2018          981.051   
               02/13/2018          672.827   
...                                    ...   
993015         07/29/2017          122.846   
               10/04/2017          409.602   
               10/08/2017          117.066   
999647         04/13/2018          661.438   
               04/18/2018         1829.295   

                               Added Sugars (by Total Sugars) (g)  \
Participant ID Date of Intake                                       
2081           08/03/2017                                  12.688   
               08/12/2017                                  15.545   
               10/25/2018                                  22.102   
11228          02/11/2018                                  26.587   
               02/13/2018                                  22.394   
...                                                           ...   
993015         07/29/2017                                   0.000   
               10/04/2017                                  33.102   
               10/08/2017                                  26.173   
999647         04/13/2018                                  18.621   
               04/18/2018                                   3.353   

                               Total Polyunsaturated Fatty Acids (PUFA) (g)  \
Participant ID Date of Intake                                                 
2081           08/03/2017                                             3.535   
               08/12/2017                                             5.010   
               10/25/2018                                             7.124   
11228          02/11/2018                                             4.190   
               02/13/2018                                             3.157   
...                                                                     ...   
993015         07/29/2017                                             3.591   
               10/04/2017                                             5.221   
               10/08/2017                                             4.210   
999647         04/13/2018                                             7.870   
               04/18/2018                                             4.283   

                               Total Monounsaturated Fatty Acids (MUFA) (g)  \
Participant ID Date of Intake                                                 
2081           08/03/2017                                             8.022   
               08/12/2017                                             9.822   
               10/25/2018                                            13.965   
11228          02/11/2018                                             3.721   
               02/13/2018                                             4.680   
...                                                                     ...   
993015         07/29/2017                                            11.981   
               10/04/2017                                            10.687   
               10/08/2017                                             8.731   
999647         04/13/2018                                            14.032   
               04/18/2018                                            13.553   

                               Refined Grains (ounce equivalents)  \
Participant ID Date of Intake                                       
2081           08/03/2017                                   0.000   
               08/12/2017                                   0.000   
               10/25/2018                                   0.000   
11228          02/11/2018                                   1.974   
               02/13/2018                                   1.23

In [9]:
def ager(date1,date2):
    # Get the age in months of each recall
    diff_time = pd.to_datetime(date1)-pd.to_datetime(date2)
    diff_time= diff_time/ np.timedelta64(1, 'M')
    return(diff_time)


In [102]:
data_dict['set_09'].columns

AttributeError: 'DataFrameGroupBy' object has no attribute 'columns'

In [106]:
data_dict['set_09'][["FRU0600"]].get_group(("Participant ID","Date of Intake"))

KeyError: ('Participant ID', 'Date of Intake')

In [108]:
groups = dict(list(data_dict['set_04']))

In [109]:
groups

{(2081,
  '08/03/2017'):      Participant ID Date of Intake  Energy (kcal)  \
 941            2081     08/03/2017         357.15   
 942            2081     08/03/2017         357.15   
 943            2081     08/03/2017         357.15   
 944            2081     08/03/2017         357.15   
 945            2081     08/03/2017         357.15   
 946            2081     08/03/2017         357.15   
 947            2081     08/03/2017         357.15   
 948            2081     08/03/2017         357.15   
 949            2081     08/03/2017         357.15   
 950            2081     08/03/2017         357.15   
 951            2081     08/03/2017         357.15   
 952            2081     08/03/2017         357.15   
 
      Total Saturated Fatty Acids (SFA) (g)  \
 941                                  7.715   
 942                                  7.715   
 943                                  7.715   
 944                                  7.715   
 945                                 

In [88]:
data_dict['set_04']

In [95]:
for key, item in data_dict['set_04']:
    print(data_dict['set_04'].get_group(key), "\n\n")

ValueError: too many values to unpack (expected 2)

In [91]:
gram2oz(data_dict['set_09']['FRU0600'])

TypeError: unsupported operand type(s) for /: 'SeriesGroupBy' and 'float'

In [90]:
def cup2oz(cup):
    cup=cup.astype('float32')
    oz=cup*8
    return(oz)

def gram2oz(gram):
    oz=gram/28.3495
    return(oz)

def T2oz(T):
    oz=T/2
    return(oz)

def grammar(key,x,num, data):
    data[key] = gram2oz(num*(data[x].astype('float').sum(axis=1)))
    return(data[key])

def from_cup(key,x,num, data):
    data[key] = cup2oz(num*(data[x].astype('float').sum(axis=1)))
    return(data[key])

def to_fluid(key, data, num):
      data[key] = num*(data[key].astype('float'))
      return(data[key])

def to_ounce(key, x, data):
    data[key] = data[x].astype('float').sum(axis=1)
    data[key] = cup2oz(data[key]/2)
    return(data[key])

def cow_stuff(key, value, data):
    x=value[:-1]
    tmp= data[x].astype('float32').sum(axis=1)
    y=value[-1]
    if key == 'hei_dairy':
        if y == 'DOT0100':
            tmp2=data[y].astype('float32')/3
            tmp3=tmp+tmp2
            data[key]=cup2oz(tmp3)
        else:
            print('NO DAIRY MISSING DOT0100, needs to be last in list')
    if key == 'hei_totproteins':
        if y == 'VEG0700':
            tmp2=data[y].astype('float32')*2 # this is normally 1/2
            data[key]=tmp+tmp2
        else:
            print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
    if key == 'hei_seafoodplantprot':
        if y == 'VEG0700':
            tmp2=data[y].astype('float32')*2
            data[key]=tmp+tmp2
        else:
            print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
    return(data[key])

In [ ]:
def make_ped_components(hei_ped_dict, complete_df, conv_dict):
    for key, value in hei_ped_dict.items():
#         print(value)
        for item in value: 
            if item in list(complete_df.columns):
                complete_df[item]=complete_df[item].astype('float32')
                if key in ['hei_fruitjuice', 'hei_SSB','formula_foz']:
#                     pdb.set_trace()
                    print(complete_df[item])
                    complete_df['%s_new'%item]=to_fluid(item, complete_df, conv_dict[key])
                    print(complete_df['%s_new'%item])
                if key in ['hei_totveg','hei_greensbeans', 'hei_wholefruit','hei_totfruit']:
                    complete_df['%s_new'%item]=to_ounce(item, value, complete_df)
                if key in ['cereal_oz', 'bbcereal_hcup']:
                    if key == 'bbcereal_hcup':
                        complete_df['%s_new'%item]=from_cup(item, value, conv_dict[key] , complete_df)
                    else:
                        x=value
                        complete_df['%s_new'%item]= complete_df[x].astype('float').sum(axis=1)
                if key in ['hei_wholegrains','hei_refinedgrains']:
                    complete_df['%s_new'%item] = complete_df[value].astype('float')
                if key in ['hei_dairy','hei_totproteins','hei_seafoodplantprot']:
                    complete_df['%s_new'%item]=cow_stuff(item, value, complete_df)
                if key in ['chocolate_candies', 'candies', 'frosting', 'sugar','nondairy_treat','baked_good', 'fries']:
                    complete_df['%s_new'%item]=grammar(item, value, conv_dict[key], complete_df)
                if key in ['sweet_sauce']:
                    x=value
                    complete_df['%s_new'%item]= T2oz(complete_df[x].astype('float').sum(axis=1))
                if key in ['syrups','Pudding','icecream']:
                    from_cup(item, value, conv_dict[key], complete_df)
                if key in ['chips','other_fried']:
                    x=value
                    complete_df['%s_new'%item]= complete_df[x].astype('float').sum(axis=1)
    return(complete_df)

In [ ]:
update_dict={'set_04':{},'set_09':{}}
for key, value in data_dict.items():
    update_dict[key]=make_ped_components(hei_ped_dict, value, conv_dict)

In [ ]:
data_dict['set_09']

In [ ]:
update_dict['set_09']['FRU0100_new']

In [ ]:
update_dict['set_09']['FRU0100']

In [ ]:
#e = HEI.diet_maker(b,c, arglist['NAMES'])
df_demo=pd.read_csv(os.path.join(basepath,arglist['XTRA']), sep=",")
# e['Participant ID'] = e['Participant ID'].astype('int32')
# df_demo['Participant ID'] = df_demo['Participant ID'].astype('int32')
# df = pd.merge(e,df_demo, on=['Participant ID'])
# df['Age_at_Intake']=DQI.ager(df['Date of Intake'], df['DoB'])
#x=DQI.BCP(df, arglist)
#y=DQI.make_ped_components(hei_ped_dict, x, conv_dict)
que=HEI.make_hei(y, make_hei_dict)
split_dict=DQI.splitter(que)
for key, value in split_dict.items():
   if key == 'DF_child':
       DQI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'child')
       DQI.check(ped_dict['child'], value, 'child', key, arglist)
   elif key == 'DF_young':
       DQI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'young')
       DQI.check(ped_dict['young'], value, 'young', key, arglist)
   else:
       DQI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'infant')
       DQI.check(ped_dict['infant'], value, 'infant', key, arglist)
